In [256]:
import json
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
data_dir="C:/Users/tsb7592/Box/Baby Toolbox Operations/Domain Teams/Language/Tobii Pilot"

In [3]:
all3= pd.read_csv('C:/Users/tsb7592/Box/Baby Toolbox Operations/Domain Teams/Language/Tobii Pilot/LWL & EF Tobii Data Export/Baby Toolbox_Data_Export_06_02_2021.tsv', sep='\t')

C:\Users\tsb7592\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (28,34,36) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
ada = pd.read_csv('C:/Users/tsb7592/Box/Baby Toolbox Operations/Domain Teams/Language/Tobii Pilot/LWL & EF Tobii Data Export/Ada LWL Only/Baby Toolbox_Data_Export_Ada_LWLonly.tsv', sep='\t')

In [5]:
data_dir2='C:/Users/tsb7592/Box/Baby Toolbox Operations/Domain Teams/Language/Tobii Pilot/LWL & EF Tobii Data Export/Second Attempts LWL Only'

In [6]:
os.chdir(data_dir2)

In [7]:
ada2= pd.read_csv('Baby Toolbox_Data_Export_Ada LWL Take 2.tsv', sep='\t') 

In [8]:
claire2= pd.read_csv('Baby Toolbox_Data_Export_Claire LWL Take 2.tsv', sep='\t') 

C:\Users\tsb7592\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (34,36) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [9]:
claire2.head()

,ExportDate,StudioVersionRec,StudioProjectName,StudioTestName,ParticipantName,RecordingName,RecordingDate,RecordingDuration,RecordingResolution,PresentationSequence,...,DistanceLeft,DistanceRight,PupilLeft,PupilRight,ValidityLeft,ValidityRight,IRMarkerCount,IRMarkerID,PupilGlassesRight,Unnamed: 173
0,6/30/2021,3.4.8,Baby Toolbox,LWL website,ClaireJune2,Rec 10,6/23/2021,303996,1920 x 1080,NaN,...,462.43,462.43,-1.00,-1.00,0.0,0.0,NaN,NaN,NaN,NaN
1,6/30/2021,3.4.8,Baby Toolbox,LWL website,ClaireJune2,Rec 10,6/23/2021,303996,1920 x 1080,NaN,...,462.35,462.35,-1.00,-1.00,0.0,0.0,NaN,NaN,NaN,NaN
2,6/30/2021,3.4.8,Baby Toolbox,LWL website,ClaireJune2,Rec 10,6/23/2021,303996,1920 x 1080,NaN,...,462.33,462.33,3.36,3.06,0.0,0.0,NaN,NaN,NaN,NaN
3,6/30/2021,3.4.8,Baby Toolbox,LWL website,ClaireJune2,Rec 10,6/23/2021,303996,1920 x 1080,NaN,...,462.14,462.14,-1.00,-1.00,0.0,0.0,NaN,NaN,NaN,NaN
4,6/30/2021,3.4.8,Baby Toolbox,LWL website,ClaireJune2,Rec 10,6/23/2021,303996,1920 x 1080,NaN,...,462.07,462.07,-1.00,-1.00,0.0,0.0,NaN,NaN,NaN,NaN


In [10]:
doc=[]
doc.append(ada)
doc.append(ada2)
doc.append(claire2)
doc.append(all3)
tdf = pd.concat(doc)

In [11]:
tdf.shape

(411928, 381)

In [64]:
lcol,rcol,wcol = k24[:3]

In [72]:
ct =tdf[tdf['ParticipantName']=='ClaireP1']
sub =ct[ct[wcol].notnull()]

In [73]:
sub[k24[:3]]

,AOI[LeftStim_Shoulder]Hit,AOI[RightStim_Hose]Hit,AOI[WholeScreen]Hit.4
30902,NaN,NaN,NaN
30903,NaN,NaN,NaN
30904,NaN,NaN,NaN
30905,NaN,NaN,NaN
30906,NaN,NaN,NaN
...,...,...,...
31859,NaN,NaN,NaN
31860,NaN,NaN,NaN
31861,NaN,NaN,NaN
31862,NaN,NaN,NaN


In [12]:
names={'P03':'Ada2', 'ClaireJune2':'ClaireJune2', 'ClaireP1':'ClaireP1', 'BrianP2':'BrianP2', 'SKJ_P2':'SKJ_P2','Adatake2':'Ada'}
tdf['ParticipantName']=tdf['ParticipantName'].map(names)

In [13]:
tdf['ParticipantName'].value_counts()

ClaireP1       107784
BrianP2        101390
SKJ_P2          96213
Ada2            36389
ClaireJune2     36145
Ada             34007
Name: ParticipantName, dtype: int64

In [14]:
keep = [col for col in tdf.columns if 'AOI' in col]
keep = [col for col in keep if 'Calib' not in col]

In [16]:
k24 = keep[4:76]

# Read json file

In [21]:
import json
import os
import pandas as pd

In [22]:
#claire
data_dir="C:/Users/tsb7592/Box/Baby Toolbox Operations/Domain Teams/Language/Tobii Pilot/Tobii Verbose Files/Claire"
os.chdir(data_dir)
df=pd.read_json('949BCA6A-D3D6-426C-9A94-E806F84E4E72@845BDD10-B9AF-4592-AA4E-0CCE76A0E8CD@00024@2021-05-11-22.11.48.356@LookListening@Claire@VerboseGazeData.json')

In [23]:
data_dir="C:/Users/tsb7592/Downloads"
os.chdir(data_dir)

In [252]:
#second attempt only
cdf=pd.read_json('2021-06-23-22.58.27.217@LookListening@Claire@VerboseGazeData.json')

In [25]:
cols=['itemID','eventName','gazeLocationName', 'gazeLocationOnScreen', 'arHitTestLookAtPointPositionX', 'arHitTestLookAtPointPositionY',"lookAtPointX", "lookAtPointY",]

# Clean the data for machine learning

In [32]:
cols=['itemID','eventName', 'gazeLocationName','gazeLocationOnScreen',
      'arHitTestLookAtPointPositionX', 'arHitTestLookAtPointPositionY',
 'lookAtPointX', 'lookAtPointY','NewLocation','NewOnScreen']

In [257]:
#Training set for Claire

def combine_tsv_json(tdf, df, person, k24):
    ct = tdf[tdf['ParticipantName']==person][k24]
    begin =list(df[df['eventName'].str.contains('presentedLiveItem')].index)
    end = list(df[df['eventName'].str.contains('completedLiveItem')].index)

    rv = []
    for i in range(24):
        lcol,rcol,wcol = k24[3*i:3*i+3]
        sub = ct[ct[wcol].notnull()]
    
        idx =[]
        mni =df.iloc[begin[i]:end[i]][['lookAtPointX','lookAtPointY']]
        mni =mni[mni['lookAtPointX'].notnull()]
        print(i, sub.shape[0], mni.shape[0])
        My_list = [*range(mni.shape[0])]
        ratio = round(sub.shape[0]/mni.shape[0], 2) - 0.03
        idx = [int(round(element * ratio)) for element in My_list]
        sub = sub.iloc[idx].reset_index()
        sub = sub[[lcol,rcol,wcol]]
        js =  mni.reset_index()
        sub.columns = ['Left','Right','Wholescreen']
        result = pd.concat([sub, js[['lookAtPointX','lookAtPointY']] ], axis =1)
        result['item'] = i
        rv.append(result)
    
    data = pd.concat(rv)
    
    #Create a Y variable base on Left, right, onscreen, offscreen
    data.loc[(data['Left'] == 1), 'Y'] = 'L'
    data.loc[(data['Right'] == 1), 'Y'] = 'R'
    data.loc[((data['Left'] == 0) & (data['Right'] ==0)& (data['Wholescreen'] ==0)), 'Y'] = 'F'
    data.loc[((data['Left'] == 0) & (data['Right'] ==0)& (data['Wholescreen'] ==1)), 'Y'] = 'O'
    
    return data

In [258]:
c1  = combine_tsv_json(tdf, df, 'ClaireP1', k24)
c2  = combine_tsv_json(tdf, cdf, 'ClaireJune2', k24)

0 1042 480
1 960 481
2 961 480
3 1041 479
4 960 481
5 1041 478
6 1040 479
7 961 480
8 960 480
9 961 480
10 1041 480
11 1041 480
12 1041 481
13 1040 480
14 1041 480
15 960 480
16 962 480
17 1040 480
18 1040 481
19 961 479
20 1040 480
21 1040 480
22 960 480
23 962 439
0 1043 479
1 1014 480
2 959 480
3 986 478
4 986 481
5 958 480
6 987 480
7 958 480
8 957 480
9 958 480
10 1043 480
11 960 479
12 987 481
13 986 479
14 958 481
15 987 480
16 985 482
17 958 480
18 987 481
19 987 479
20 986 481
21 986 481
22 958 480
23 959 481


In [259]:
c1.Y.value_counts()

L    5776
R    4186
F    1103
O     413
Name: Y, dtype: int64

In [261]:
# example of multi-class classification task
from numpy import where
from collections import Counter
from sklearn.datasets import make_blobs
from matplotlib import pyplot

from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

X = c1[['lookAtPointX','lookAtPointY']] 
y = c1['Y']

In [262]:
from sklearn import metrics
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state = True, test_size = 0.25) 
y_pred =OneVsRestClassifier(LinearSVC(random_state=0)).fit(x_train, y_train).predict(x_test)

# Print the precision and recall, among other metrics
print(metrics.classification_report(y_test, y_pred, digits=3))

              precision    recall  f1-score   support

           F      0.000     0.000     0.000       273
           L      0.687     0.858     0.763      1433
           O      0.000     0.000     0.000       120
           R      0.694     0.718     0.706      1044

    accuracy                          0.690      2870
   macro avg      0.345     0.394     0.367      2870
weighted avg      0.595     0.690     0.638      2870



In [264]:
x_train= c1[['lookAtPointX','lookAtPointY']] 
y_train = c1['Y']
x_test= c2[['lookAtPointX','lookAtPointY']] 
y_test = c2['Y']

y_pred =OneVsRestClassifier(LinearSVC(random_state=0)).fit(x_train, y_train).predict(x_test)

# Print the precision and recall, among other metrics
print(metrics.classification_report(y_test, y_pred, digits=3))

              precision    recall  f1-score   support

           F      0.000     0.000     0.000      1361
           L      0.670     0.844     0.747      4985
           O      0.000     0.000     0.000       186
           R      0.742     0.779     0.760      4991

    accuracy                          0.703     11523
   macro avg      0.353     0.406     0.377     11523
weighted avg      0.611     0.703     0.652     11523



In [265]:
#Sheila
data_dir="C:/Users/tsb7592/Box/Baby Toolbox Operations/Domain Teams/Language/Tobii Pilot/Tobii Verbose Files/Sheila"
os.chdir(data_dir)
sdf=pd.read_json('B8C654C6-5EAA-452E-8A80-B0281F7533C8@845BDD10-B9AF-4592-AA4E-0CCE76A0E8CD@00030@2021-05-12-00.31.22.201@LookListening@SKJ@VerboseGazeData.json')

In [267]:
s1  = combine_tsv_json(tdf, sdf, 'SKJ_P2', k24)

0 998 480
1 998 480
2 997 479
3 997 480
4 998 480
5 998 480
6 998 480
7 926 479
8 997 480
9 927 479
10 998 480
11 997 480
12 997 480
13 926 480
14 998 479
15 998 480
16 926 480
17 997 480
18 998 479
19 998 479
20 998 480
21 997 481
22 997 479
23 997 480


In [268]:
X = s1[['lookAtPointX','lookAtPointY']] 
y = s1['Y']

x_train, x_test, y_train, y_test = train_test_split(X, y, random_state = True, test_size = 0.25) 
y_pred =OneVsRestClassifier(LinearSVC(random_state=0)).fit(x_train, y_train).predict(x_test)

# Print the precision and recall, among other metrics
print(metrics.classification_report(y_test, y_pred, digits=3))

              precision    recall  f1-score   support

           F      0.000     0.000     0.000       240
           L      0.781     0.855     0.816      1331
           O      0.000     0.000     0.000       109
           R      0.731     0.867     0.793      1199

    accuracy                          0.756      2879
   macro avg      0.378     0.430     0.402      2879
weighted avg      0.665     0.756     0.708      2879



In [274]:
ppl

{'Ada':      left  right  onScreenAway  offScreenAway
 0   381.0  127.0          61.0          376.0
 1   185.0  130.0          46.0          640.0
 2   204.0  103.0          71.0          623.0
 3    34.0  430.0          78.0          482.0
 4   154.0  486.0           1.0          305.0
 5    50.0   73.0          32.0          792.0
 6    62.0   41.0         199.0          698.0
 7   217.0  282.0          61.0          412.0
 8    99.0  369.0          10.0          468.0
 9   115.0   85.0          87.0          658.0
 10  199.0  102.0           0.0          672.0
 11  334.0  324.0           3.0          338.0
 12  121.0   32.0          99.0          697.0
 13  112.0  274.0          26.0          561.0
 14   56.0  369.0          38.0          510.0
 15  158.0  372.0           0.0          443.0
 16  227.0  500.0          11.0          208.0
 17    0.0   92.0          11.0          844.0
 18  151.0    0.0          30.0          819.0
 19   11.0  103.0          48.0          812.0
 20  2

In [275]:
#Ada
data_dir="C:/Users/tsb7592/Downloads"
os.chdir(data_dir)
adf=pd.read_json('Ada@VerboseGazeData.json')
adf2=pd.read_json('2021-06-24-12.42.35.513@LookListening@Ada@VerboseGazeData 2.json')
a1  = combine_tsv_json(tdf, adf, 'Ada', k24)
a2  = combine_tsv_json(tdf, adf2, 'Ada2', k24)

0 945 458
1 1001 275
2 1001 293
3 1024 480
4 946 428
5 947 337
6 1000 369
7 972 480
8 946 479
9 945 479
10 973 303
11 999 424
12 949 480
13 973 480
14 973 396
15 973 263
16 946 480
17 947 437
18 1000 480
19 974 480
20 973 481
21 947 479
22 976 381
23 973 469
0 1051 482
1 999 297
2 1025 481
3 974 382
4 1053 481
5 1000 407
6 971 479
7 973 256
8 1025 425
9 999 334
10 970 480
11 972 359
12 971 479
13 999 441
14 972 325
15 997 480
16 970 481
17 1026 479
18 972 481
19 1000 403
20 970 480
21 1025 479
22 1000 472
23 971 359


In [277]:
x_train= a1[['lookAtPointX','lookAtPointY']] 
y_train = a1['Y']
x_test= a2[['lookAtPointX','lookAtPointY']] 
y_test = a2['Y']

x_train, x_test, y_train, y_test = train_test_split(X, y, random_state = True, test_size = 0.25) 
y_pred =OneVsRestClassifier(LinearSVC(random_state=0)).fit(x_train, y_train).predict(x_test)

# Print the precision and recall, among other metrics
print(metrics.classification_report(y_test, y_pred, digits=3))

              precision    recall  f1-score   support

           F      0.000     0.000     0.000      1071
           L      0.449     0.493     0.470       712
           O      0.025     0.047     0.033        86
           R      0.391     0.921     0.549       687

    accuracy                          0.387      2556
   macro avg      0.216     0.365     0.263      2556
weighted avg      0.231     0.387     0.280      2556



In [266]:
#Brian
data_dir="C:/Users/tsb7592/Box/Baby Toolbox Operations/Domain Teams/Language/Tobii Pilot/Tobii Verbose Files/Brian"
os.chdir(data_dir)
bdf=pd.read_json('A87CEC62-214F-4B81-A3CB-45B1DA445644@845BDD10-B9AF-4592-AA4E-0CCE76A0E8CD@00026@2021-05-12-00.13.17.121@LookListening@Brian@VerboseGazeData.json')

In [269]:
b1  = combine_tsv_json(tdf, bdf, 'BrianP2', k24)

0 971 480
1 1046 479
2 971 480
3 971 480
4 971 481
5 971 480
6 971 479
7 971 480
8 971 482
9 971 480
10 971 480
11 971 480
12 1046 480
13 971 479
14 972 481
15 971 481
16 1046 480
17 972 480
18 971 479
19 971 480
20 971 481
21 971 480
22 972 480
23 1046 481


In [270]:
X = b1[['lookAtPointX','lookAtPointY']] 
y = b1['Y']

x_train, x_test, y_train, y_test = train_test_split(X, y, random_state = True, test_size = 0.25) 
y_pred =OneVsRestClassifier(LinearSVC(random_state=0)).fit(x_train, y_train).predict(x_test)

# Print the precision and recall, among other metrics
print(metrics.classification_report(y_test, y_pred, digits=3))

              precision    recall  f1-score   support

           F      0.000     0.000     0.000        41
           L      0.809     0.890     0.848      1214
           O      0.804     0.560     0.660       536
           R      0.858     0.898     0.878      1090

    accuracy                          0.819      2881
   macro avg      0.618     0.587     0.596      2881
weighted avg      0.815     0.819     0.812      2881

